In [5]:
import os
import requests
from openai import OpenAI
import anthropic
from dotenv import load_dotenv
import gradio as gr

In [6]:

load_dotenv(override=True)

OPENAI_KEY = os.getenv('OPENAI_API_KEY')
CLAUDE_KEY = os.getenv('ANTHROPIC_API_KEY')

if OPENAI_KEY:
    print(f"OpenAI API Key exists and begins {OPENAI_KEY[:8]}")
else:
    print("OpenAI API Key not set")
    
if CLAUDE_KEY:
    print(f"Anthropic API Key exists and begins {CLAUDE_KEY[:7]}")
else:
    print("Anthropic API Key not set")


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [7]:
openai = OpenAI()
claude = anthropic.Anthropic()

gpt_model = 'gpt-4o-mini'
claude_model = 'claude-3-haiku-20240307'

In [8]:
system_prompt = 'You are an AI assistant that writes in Markdown'

In [9]:
def stream_gpt(message):
    stream = openai.chat.completions.create(
        messages = [{"role" : "system", "content" : system_prompt},
                   {"role" : "user", "content" : message}],
        stream=True,
        model = gpt_model
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [10]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model=claude_model,
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [11]:
def call_llm(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=='Claude':
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result
    

In [12]:
view = gr.Interface(
    fn=call_llm,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)

view.launch()


* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
